In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

<h1> Import dataset </h1>

In [2]:
data = pd.read_csv('dataset.csv')

In [3]:
data.head()

,local_ref_1,vn_marital_status,resid_province,resid_district,resid_wards,birth_incorp_date,amount_week,max_amount_week,min_amount_week,distinct_payment_code_week,...,distinct_trans_group_3month,distinct_ref_no_3month,most_act_mar2021_count,most_act_mar2021,total_act_mar2021,total_amt_mar2021,most_act_juin2021_count,most_act_juin2021,total_act_juin2021,total_amt_juin2021
0,MALE,Married,27.0,262.0,9436.0,1992.0,NaN,NaN,NaN,0.0,...,2,33,5.0,Giao dich chuyen khoan,5.0,0.000016,14.0,Giao dich chuyen khoan,17.0,0.006543
1,MALE,Single,79.0,769.0,27097.0,1986.0,NaN,NaN,NaN,NaN,...,2,5,NaN,NaN,NaN,NaN,1.0,Giao dich thanh toan,1.0,0.000062
2,MALE,Married,1.0,250.0,8989.0,1980.0,NaN,NaN,NaN,NaN,...,2,7,4.0,Giao dich chuyen khoan,6.0,0.000046,1.0,Giao dich chuyen khoan,2.0,0.000071
3,MALE,Single,40.0,431.0,18055.0,2001.0,NaN,NaN,NaN,NaN,...,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MALE,Married,49.0,512.0,20758.0,1989.0,0.0,0.000114,0.000636,0.0,...,3,24,9.0,Giao dich chuyen khoan,10.0,0.000170,17.0,Giao dich chuyen khoan,19.0,0.000575


In [4]:
#Lọc ra những KH có giao dịch trong cả 1 tháng gần nhất và trong 3 tháng gần nhất
data.dropna(axis = 0, subset = ['amount_month', 'amount_3month'], inplace = True)

In [5]:
#Xếp loại các khách hàng churn và active
data["Is_churn"] = np.where(
   (data['total_amt_mar2021'].isna() == True) & (data['total_amt_juin2021'].isna() == True), 1, 0)

In [6]:
data['Is_churn'].value_counts()

0    70628
1     6372
Name: Is_churn, dtype: int64

In [7]:
#loại các cột liên quan đến tháng 3 và tháng 6
data = data.drop(['most_act_mar2021_count', 'most_act_mar2021', 'total_act_mar2021', 'total_amt_mar2021',
           'most_act_juin2021_count', 'most_act_juin2021', 'total_act_juin2021', 'total_amt_juin2021'], axis = 1)

<h1> Handing missing data </h1>

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77000 entries, 0 to 99997
Data columns (total 28 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   local_ref_1                   74607 non-null  object 
 1   vn_marital_status             74357 non-null  object 
 2   resid_province                71332 non-null  float64
 3   resid_district                71332 non-null  float64
 4   resid_wards                   71330 non-null  float64
 5   birth_incorp_date             76992 non-null  float64
 6   amount_week                   54800 non-null  float64
 7   max_amount_week               54800 non-null  float64
 8   min_amount_week               54800 non-null  float64
 9   distinct_payment_code_week    55345 non-null  float64
 10  count_payment_code_week       55345 non-null  float64
 11  distinct_trans_group_week     55345 non-null  float64
 12  distinct_ref_no_week          55345 non-null  float64
 13  a

Cột 'amount_week' đang chứa 22200 missing data. Các giá trị null này thể hiện khách hàng không có giao dịch gì trong 1 tuần trước đó. Do đó, chúng ta sẽ delete những dòng null này.

In [9]:
data = data[data['amount_week'].notna()]

Chúng ta sẽ kiểm tra lại dataset có những cột nào chứa missing data

In [10]:
missing_data = data.isnull()
for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("")   

local_ref_1
False    53149
True      1651
Name: local_ref_1, dtype: int64

vn_marital_status
False    52982
True      1818
Name: vn_marital_status, dtype: int64

resid_province
False    50988
True      3812
Name: resid_province, dtype: int64

resid_district
False    50991
True      3809
Name: resid_district, dtype: int64

resid_wards
False    50991
True      3809
Name: resid_wards, dtype: int64

birth_incorp_date
False    54796
True         4
Name: birth_incorp_date, dtype: int64

amount_week
False    54800
Name: amount_week, dtype: int64

max_amount_week
False    54800
Name: max_amount_week, dtype: int64

min_amount_week
False    54800
Name: min_amount_week, dtype: int64

distinct_payment_code_week
False    54800
Name: distinct_payment_code_week, dtype: int64

count_payment_code_week
False    54800
Name: count_payment_code_week, dtype: int64

distinct_trans_group_week
False    54800
Name: distinct_trans_group_week, dtype: int64

distinct_ref_no_week
False    54800
Name: distinct_ref_n

Như vậy, chúng ta hiện còn 6 cột chứa missing value:
<ol>
    <li>"local_ref_1": 1651 missing data
    <li>"vn_marital_status": 1818 missing data
    <li>"resid_province": 3812 missing data
    <li>:"resid_district": 3809 missing data
    <li>:"resid_wards": 3809 missing data
    <li>:"birth_incorp_date": 4 missing data

<h2> Replace missing values of 'birth_incorp_date' column by mean </h2>

In [11]:
data['birth_incorp_date'] = 2020 - data[['birth_incorp_date']]

In [12]:
pd.options.display.max_rows = 999
data['birth_incorp_date'].value_counts()

28.0     3033
30.0     3008
29.0     2978
27.0     2870
26.0     2814
31.0     2601
32.0     2480
25.0     2359
33.0     2307
35.0     2285
24.0     2148
36.0     2102
34.0     2040
37.0     1988
23.0     1984
38.0     1801
39.0     1386
22.0     1379
40.0     1180
41.0     1076
21.0     1013
42.0      892
43.0      876
20.0      841
44.0      819
18.0      812
19.0      764
45.0      682
46.0      601
47.0      513
48.0      459
49.0      418
50.0      335
17.0      297
51.0      235
52.0      218
16.0      165
53.0      160
54.0      154
55.0      106
56.0       93
57.0       85
58.0       64
60.0       61
59.0       57
15.0       52
61.0       46
62.0       39
64.0       27
63.0       22
66.0       16
65.0        8
69.0        8
71.0        4
10.0        4
67.0        4
11.0        3
1.0         3
70.0        3
68.0        2
2.0         2
75.0        2
82.0        1
13.0        1
947.0       1
12.0        1
3.0         1
76.0        1
80.0        1
5.0         1
88.0        1
241.0 

In [13]:
data1 = data.loc[(data['birth_incorp_date'] < 101) & (data['birth_incorp_date'] > 5)]

In [14]:
avg_age = data1['birth_incorp_date'].mean(axis=0)
avg_age

32.024477056182235

In [15]:
data['birth_incorp_date'].replace(np.nan, 33, inplace = True)

Chúng ta chỉ quan sát những khách hàng trong độ tuổi từ 6 đến 100 tuổi.

In [16]:
data = data.loc[(data['birth_incorp_date'] < 101) & (data['birth_incorp_date'] > 5)]
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54790 entries, 4 to 99993
Data columns (total 28 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   local_ref_1                   53140 non-null  object 
 1   vn_marital_status             52973 non-null  object 
 2   resid_province                50980 non-null  float64
 3   resid_district                50983 non-null  float64
 4   resid_wards                   50983 non-null  float64
 5   birth_incorp_date             54790 non-null  float64
 6   amount_week                   54790 non-null  float64
 7   max_amount_week               54790 non-null  float64
 8   min_amount_week               54790 non-null  float64
 9   distinct_payment_code_week    54790 non-null  float64
 10  count_payment_code_week       54790 non-null  float64
 11  distinct_trans_group_week     54790 non-null  float64
 12  distinct_ref_no_week          54790 non-null  float64
 13  a

<h2> Handling missing data in 'vn_marital_status' column </h2>

In [17]:
data['age_group'] = 0

In [18]:
def impute_age_group(col):
    birth_incorp_date = col[0]
    if birth_incorp_date < 19:
        return '6_18'
    if birth_incorp_date < 41:
        return '19_40'
    else:
        return 'over 40'

In [19]:
data['age_group'] = data[['birth_incorp_date']].apply(impute_age_group, axis = 1)

In [19]:
data['test_column'] = 1
data_grtest = data[['vn_marital_status', 'age_group', 'test_column']]
group_test = data_grtest.groupby(['vn_marital_status', 'age_group'], as_index = False).sum()
group_test.sort_values('age_group')

,vn_marital_status,age_group,test_column
0,Divorced,19_40,12
2,Married,19_40,19807
5,Other,19_40,194
8,Partner,19_40,19
10,Seperated,19_40,2
12,Single,19_40,24000
15,Widow,19_40,2
3,Married,6_18,34
6,Other,6_18,1
13,Single,6_18,1296


Từ dữ kiện trên, chúng ts sẽ fill 'Single' đối với những khách hàng có age_group là '19-40'; 'Married' đối với những khách hàng có age_group là 'over 40'; 'Single' đối với những khách hàng có age_group là '6-18'.

In [20]:
def impute_marrital_status(cols):
    vn_marital_status = cols[0]
    age_group = cols[1]
    if pd.isnull(vn_marital_status):
        if age_group == '6-18':
            return 'Single'
        if age_group == '19-40':
            return 'Single'
        else:
            return 'Married'
    else:
        return vn_marital_status

In [21]:
data['vn_marital_status'] = data[['vn_marital_status', 'age_group']].apply(impute_marrital_status, axis = 1)

<h2> Handling missing data in 'local_ref_1' column </h2>

Chúng ta sẽ fill missing values của cột 'local_ref_1' bằng frequent value dựa trên cơ sở 2 cột 'vn_marital_status' và 'age_group'.

In [22]:
data_grtest1 = data[['local_ref_1','vn_marital_status', 'age_group', 'test_column']]
group_test1 = data_grtest1.groupby(['local_ref_1','vn_marital_status', 'age_group'], as_index = False).sum()
group_test1.sort_values([ 'age_group', 'vn_marital_status'])

,local_ref_1,vn_marital_status,age_group,test_column
0,FEMALE,Divorced,19_40,7
14,MALE,Divorced,19_40,5
2,FEMALE,Married,19_40,7856
16,MALE,Married,19_40,12100
5,FEMALE,Other,19_40,88
19,MALE,Other,19_40,106
7,FEMALE,Partner,19_40,3
22,MALE,Partner,19_40,16
8,FEMALE,Seperated,19_40,2
10,FEMALE,Single,19_40,8428


In [23]:
def impute_sex(cols):
    sex = cols[0]
    age_group = cols[1]
    marital_status = cols[2]
    if pd.isnull(sex):
        if age_group == '6-18':
            if marital_status == 'Single':
                return 'FEMALE'
            else:
                return 'MALE'
        if age_group == '19-40':
            if marital_status == 'Divorced':
                return 'FEMALE'
            if marital_status == 'Married':
                return 'MALE'
            if marital_status == 'Other':
                return 'MALE'
            if marital_status == 'Partner':
                return 'MALE'
            if marital_status == 'Seperated':
                return 'FEMALE'
            else:
                return 'MALE'
        else:
            if marital_status == 'Divorced':
                return 'MALE'
            if marital_status == 'Married':
                return 'MALE'
            if marital_status == 'Other':
                return 'MALE'
            if marital_status == 'Partner':
                return 'MALE'
            if marital_status == 'Seperated':
                return 'FEMALE'
            else:
                return 'MALE'
    else:
        return sex

In [24]:
data['local_ref_1'] = data[['local_ref_1', 'age_group', 'vn_marital_status']].apply(impute_sex, axis = 1)

<h2> Handing missing data in 'resid_province' column by mode</h2>

In [25]:
# Tìm frequent value trong cột 'resid_provice'
data['resid_province'].value_counts().idxmax()

1.0

In [26]:
# Thay các giá trị NaN bằng '1'
data['resid_province'].replace(np.nan, 1, inplace = True)

Chúng ta không tìm hiểu mối quan hệ với 2 trường 'resid_district' và 'resid_wards'.

In [27]:
data.drop(['resid_district', 'resid_wards'], axis = 1, inplace = True)

In [28]:
data.drop(['test_column', 'age_group'], axis = 1, inplace = True)

Note:
- Tìm khách hàng churn phần lớn trong range tuổi nào.
        data_grtest2 = data[['birth_incorp_date', 'Is_churn', 'test_column']]
        group_test2 = data_grtest2.groupby(['birth_incorp_date', 'Is_churn'], as_index = False).sum()
        group_test2.sort_values('Is_churn')
- Tạo 1 group_age mới từ range age vừa tìm được.
        def impute_age_group(col):
            birth_incorp_date = col[0]
            if birth_incorp_date < 16:
                 return '6_15'
            if birth_incorp_date < 46:
                 return '16_45'
            else:
                 return 'over 45'
        data['age_group'] = data[['birth_incorp_date']].apply(impute_age_group, axis = 1)
- Xem tỷ lệ churn trên từng range tuổi.
         data_grtest3 = data[['age_group', 'Is_churn', 'test_column']]
         group_test3 = data_grtest3.groupby(['age_group', 'Is_churn'], as_index = False).sum()
         group_test3.sort_values('Is_churn')

In [29]:
# Tạo 1 biến 'diff_amount_month' là sự chênh lệch giữa lần giao dịch lớn nhất và giao dịch nhỏ nhất trong 1 tháng trước đó
data['diff_amount_month'] = data['max_amount_month'] - data['min_amount_month']

In [30]:
# Tạo 1 biến 'diff_amount_week' là sự chênh lệch giữa lần giao dịch lớn nhất và giao dịch nhỏ nhất trong 1 tuần trước đó
data['diff_amount_week'] = data['max_amount_week'] - data['min_amount_week']

In [31]:
# Tạo 1 biến 'diff_amount_3month' là sự chênh lệch giữa lần giao dịch lớn nhất và giao dịch nhỏ nhất trong 3 tháng trước đó
data['diff_amount_3month'] = data['max_amount_3month'] - data['min_amount_3month']

In [32]:
data.to_csv('clean_dataset.csv', index=False)

In [33]:
data

,local_ref_1,vn_marital_status,resid_province,birth_incorp_date,amount_week,max_amount_week,min_amount_week,distinct_payment_code_week,count_payment_code_week,distinct_trans_group_week,...,max_amount_3month,min_amount_3month,distinct_payment_code_3month,count_payment_code_3month,distinct_trans_group_3month,distinct_ref_no_3month,Is_churn,diff_amount_month,diff_amount_week,diff_amount_3month
4,MALE,Married,49.0,31.0,0.0,0.000114,0.000636,0.0,0.0,1.0,...,0.000380,5.920000e-07,1,11,3,24,0,2.461670e-04,-0.000522,0.000379
7,MALE,Married,38.0,28.0,0.0,0.000476,0.000011,0.0,0.0,1.0,...,0.000300,5.920000e-07,1,2,3,141,0,5.684690e-04,0.000464,0.000299
12,MALE,Single,56.0,24.0,0.0,0.000014,0.000079,0.0,0.0,1.0,...,0.000026,5.920000e-07,1,6,3,20,0,1.334000e-05,-0.000065,0.000025
17,FEMALE,Single,92.0,30.0,0.0,0.000449,0.001135,0.0,0.0,2.0,...,0.003800,2.960000e-06,2,10,5,79,0,4.460200e-04,-0.000686,0.003797
18,MALE,Single,38.0,38.0,0.0,0.001429,0.000681,0.0,0.0,1.0,...,0.001230,5.920000e-06,0,0,3,62,0,1.422651e-03,0.000747,0.001224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99956,MALE,Married,83.0,30.0,0.0,0.000031,0.000172,0.0,0.0,1.0,...,0.000015,3.070000e-05,0,0,1,2,0,3.000000e-07,-0.000141,-0.000016
99976,FEMALE,Single,33.0,29.0,0.0,0.000408,0.002271,0.0,0.0,1.0,...,0.000200,5.915760e-04,0,0,1,1,0,-1.834130e-04,-0.001863,-0.000392
99980,MALE,Single,70.0,17.0,0.0,0.000036,0.000203,0.0,0.0,1.0,...,0.000018,5.290000e-05,0,0,1,1,0,-1.640000e-05,-0.000167,-0.000035
99985,MALE,Single,1.0,35.0,0.0,0.000008,0.000045,0.0,0.0,1.0,...,0.000004,1.180000e-05,0,0,1,1,0,-3.640000e-06,-0.000037,-0.000008
